In [38]:
from matplotlib import pyplot as plt
import numpy as np

c = 3e8 # m/s
cavity_length = 15e-3  # let's assume cavity length is 15 mm

wavelengths = [375e-9, 397e-9, 423e-9, 639e-9, 865e-9]
gratings = [1200, 1800, 2400, 3600]  # lines/mm

def rad2deg(rad):
    return rad * 180/np.pi


### Diffration grating equation

\begin{equation}
d(\sin(\theta_m) + \sin(\theta_i)) = m\lambda
\end{equation}

where $d$ is diffraction grating period, $\theta_m$ is an angle of $m-th$ diffraction order, $\theta_i$ is an angle of the incident beam

Littrow configuration is often used in external cavity laser systems, which means that beam of a particular diffraction order is reflected back to the cavityy. Most often $m=1$ is used as an optical feedback due to various reasons. This means that $\theta_{m=1}$ is equal to $\theta_i$ which results in:
\begin{equation}
2d\sin(\theta_{m=1}) = m\lambda
\end{equation}

and allows to obtain equation of $\theta$ as a function of $\lambda$ and $d$:
\begin{equation}
\theta(\lambda,d)_{m=1} = \arcsin(\frac{\lambda}{2d})
\end{equation}

Which denotes diffraction grating's theoretically available angles that allow to obtain ECDL

In [39]:

def find_littrow_angle(grating_period, lambda0, m=1):
    d = grating_period
    
    # for small radians we could assume that deviation range can be calculated without lambda0
    # however, for 639 nm, arcsin's argument is 0.58, which is not small enough
    if m*lambda0/(2*d) > 1:
        print(f"[Warning]: arcsin argument is greater than 1: {m*lambda0/(2*d)}")
        raise ValueError("arcsin argument is greater than 1")
    
    central_angle = np.arcsin((m*lambda0/(2*d)))
    return central_angle


### Angular dispersion

Angular dispersion is defined as
\begin{equation}
D = \frac{d\theta}{d\lambda}
\end{equation}

Grating equation for Littrow configuration, where incident beam angle is same as the diffracted one:
\begin{equation}
m\lambda = 2a\sin(\theta)
\end{equation}

We can now differentiate both sides of the equation with respect to $\theta$ (note that $\theta$ is a function of $\lambda$):
\begin{equation}
m = 2a\frac{d\theta}{d\lambda}\cos(\theta)
\end{equation}

\begin{equation}
\frac{d\theta}{d\lambda} = \frac{m}{2a\cos(\theta)}
\end{equation}

However, this might give false impression that the ratio $\frac{m}{a}$ maybe chosen independently of other paremeters. Let's take $\frac{m}{a}$ ratio from grating equation, because this condition must still be met:

\begin{equation}
\frac{m}{a} = \frac{2\sin(\theta)}{\lambda},
\end{equation}
so after substitution:
\begin{equation}
\frac{d\theta}{d\lambda} = \frac{\sin(\theta)}{\lambda\cos(\theta)} = \frac{1}{\lambda}\tan{\theta}
\end{equation}

### FSR
Free spectral range is a difference between neighbouring longitudonal modes that are allowed to emerge
in the resonator. It may be approximated as:

\begin{equation}
FSR=\frac{c}{2nL}
\end{equation}
where $L$ is the length of the cavity resonator and $n$ is the refraction index (for air it may be set to 1). 

In [42]:
def ang_dispersion(theta_m, lambda0):
    # returns dispersion in deg/m
    # allows to retrieve angle change as a function of wavelength
    # use 1/D_littrow to get change of wavelength as a function of angle
    D_littrow = np.tan(theta_m)/lambda0

    return D_littrow

def find_linear_range(deviation_angle, cavity_length):
    # linear range at the cavity_length distance from the grating
    # 2 is because we are calculating the range from the center
    return np.tan(deviation_angle) * cavity_length * 2


def littrow_params_ecdl(grating_period, lambda0, lambda_deviation, cavity_length, n_FSR_in_range, m=1):
    try:
        central_angle = find_littrow_angle(grating_period, lambda0, m)
    except ValueError as e:
        raise e
    # FIXME: deviation actually means something in laser systems, not sure if used in this meaning is correct
    deviation_angle = ang_dispersion(central_angle, lambda0) * lambda_deviation
    
    linear_range = find_linear_range(deviation_angle, cavity_length)
    linear_separation = linear_range/n_FSR_in_range # this is more or less
    
    return central_angle, deviation_angle, linear_range, linear_separation

In [44]:
for lambda0 in wavelengths:

    # let's assume that the wavelength deviation is 1 nm for all the wavelengths
    # just for the sake of calculations and dispersion comparison
    lambda0_dev = 1e-9
    freq0 = c/lambda0
    freq0_start = c/(lambda0+lambda0_dev)
    freq0_end = c/(lambda0-lambda0_dev)
    freq_range = freq0_end - freq0_start
    FSR = c/(2*cavity_length)
    n_allowed_mods = freq_range / FSR

    print("_"*80)    
    print(f"=== Central Wavelegnth: {lambda0*1e9:.2f} nm ===")
    print(f"Frequency: {freq0/1e12:.2f} THz +/- {freq_range/1e12:.2f} THz")
    print(f"FSR: {FSR/1e9:.2f} GHz")
    print(f"Number of allowed mods in range: {n_allowed_mods:.2f}")
    print()
    for grating in gratings:
        grating_period = 1e-3/grating
        try:
            central_angle, deviation_angle, linear_range, linear_separation = littrow_params_ecdl(grating_period, lambda0, lambda0_dev, cavity_length, n_allowed_mods)
        except ValueError:
            print(f"[!!!] Skipping grating: {grating} for lambda0: {lambda0*1e9:.2f} nm")
            print()
            continue

        disp = ang_dispersion(central_angle, lambda0)

        print(f"===> Grating lines/mm: {grating} period: {grating_period*1e6:.4} um")
        print(f"\tMain angle: {rad2deg(central_angle):.4f} deg +/- {rad2deg(deviation_angle):.4f} deg = {rad2deg(deviation_angle)*3600:.4f} arcsec")
        print(f"\tAngular dispersion [rad/nm]: {disp/1e9}; [deg/nm]: {rad2deg(disp)/1e9}")
        print(f"\tLinear range at `cavity_length` ({cavity_length*1e3} mm) distance from the grating: {linear_range*1e6:.4f} um")
        print("\tLinear separation between mods at `cavity_length` distance:")
        print(f"\t\t[um]: {linear_separation*1e6:.6f}")
        print(f"\t\t[arcsec]: {rad2deg(deviation_angle)*3600/n_allowed_mods:.6f}")
        print()



________________________________________________________________________________
=== Central Wavelegnth: 375.00 nm ===
Frequency: 800.00 THz +/- 4.27 THz
FSR: 10.00 GHz
Number of allowed mods in range: 426.67

===> Grating lines/mm: 1200 period: 0.8333 um
	Main angle: 13.0029 deg +/- 0.0353 deg = 127.0157 arcsec
	Angular dispersion [rad/nm]: 0.0006157896069489425; [deg/nm]: 0.035282145546194243
	Linear range at `cavity_length` (15.0 mm) distance from the grating: 18.4737 um
	Linear separation between mods at `cavity_length` distance:
		[um]: 0.043297
		[arcsec]: 0.297691

===> Grating lines/mm: 1800 period: 0.5556 um
	Main angle: 19.7246 deg +/- 0.0548 deg = 197.2095 arcsec
	Angular dispersion [rad/nm]: 0.0009560985791406336; [deg/nm]: 0.05478041338321303
	Linear range at `cavity_length` (15.0 mm) distance from the grating: 28.6830 um
	Linear separation between mods at `cavity_length` distance:
		[um]: 0.067225
		[arcsec]: 0.462206

===> Grating lines/mm: 2400 period: 0.4167 um
	Main a

### Mode hopping

One can substitute $\lambda$ in grating equation with $\frac{c}{\nu}$ to obtain
frequency that is retro-fitted back to the cavity:

\begin{equation}
\nu_g = \frac{c}{2d\sin(\theta)}
\end{equation}

A mode hop occurs when the difference between the modal and grating frequencies 
exceeds half of the FSR, i.e.:

\begin{equation}
|\nu_g - \nu_m| >= \frac{1}{2}\frac{c}{2L}
\end{equation}

### WIP Resolution
Rayleigh criterion states that two different wavelengths $\lambda_1$ and $\lambda_2$ can be resolved if maksimum of one line is in the same spot as the first minum of the second.
$\delta\lambda$ or $(\Delta\lambda)_{min}$ is the least resolvable wavelength difference, or limit of resolution. At the limit of resolution the angular separation is half the linewidth.

Chromatic resolving power is defined as:
\begin{equation}
\mathscr{R} \equiv \frac{\lambda}{\delta\lambda}
\end{equation}

Theoretical resolving power of a planar diffraction grating is given by:
\begin{equation}
\mathscr{R} = mN
\end{equation}

One can use the grating equation to replace m:
\begin{equation}
\mathscr{R} = \frac{Na(\sin(\theta_m)+\sin(\theta_i))}{\lambda}
\end{equation}

That means that the resolving power cannot exceed $\frac{2Na}{\lambda}$ which occurs when both angles are the same ($\theta_m=\theta_in=90^{\circ}$). Greatest values of resolving power are obtained when the grating is used in an autocollimation (Littrow configuration):
\begin{equation}
\mathscr{R} = \frac{2Na\sin(\theta_i)}{\lambda}
\end{equation}


Angular width of the line:
\begin{equation}
\Delta\theta = \frac{2\lambda}{Na\cos(\theta_m)}
\end{equation}
where $N$ is number of lines covered by the beam. The angular linewidth varies inveries with the width of the grating itself $Na$.

, or:
\begin{equation}
\delta\theta = \frac{\lambda}{Na\cos{\theta_m}}
\end{equation}

\begin{equation}
\delta = \frac{\lambda^2}{Na(\sin(\theta_i) + \sin(\theta_m))}
\end{equation}



\begin{equation}
\mathscr{R} = \frac{Na(\sin(\theta_i)+\sin(\theta_m))}{\lambda}
\end{equation}



### WIP Wavelength
Emitted wavelength is given with 
\begin{equation}
\lambda = \frac{2nl}{m}
\end{equation}

where $m$ is the longitudonal mode number, $l$ is laser's cavity length and $n$ is refractive index of the diode material (oftentimes $n=0.3$)



